In [2]:
#include <iostream>
#include <exception>
#include <stdexcept> // stdexcept header contains runtime_error
using namespace std;

# Exceptions

## Intro

- An **exception** is an indication of a problem that occurs during a program's execution.
- Exception handling enables you to create applications that can handle exceptions and perform appropriate cleanup when an exception that cannot or should not be handled occurs.
- In many cases, this allows a program to continue executing as if no problem had been encountered.
- The features presented in this chapter enable you to write **robust** and **fault-tolerant** programs that can deal with problems that may arise and continue executing or terminate gracefully.

## Software Engineering

- Exceptions provide a way to react to exceptional circumstances (like runtime errors) in programs by transferring control to special functions called **handlers**.
- Exception handling provides a standard mechanism for processing errors.
    - This is especially important when working on a large project.
- Without exception handling, it's common for a function to calculate and return a value on success or return an error indicator on failure.
    - A common problem with this architecture is using the return value in a subsequent calculation without first checking whether the value is the error indicator.
    - Exception handling eliminates this problem.    

## Example: Memory Allocation Exception 

- Exception thrown on failure allocating memory
    - Type of the exceptions thrown by the standard definitions of operator `new` and operator `new[]` when they fail to allocate the requested storage space.


In [3]:
{
    int* myarray= new int[1000000000000];
    cout << "Done!";
}

Standard Exception: std::bad_alloc

## Exception Handling Workflow

- To catch exceptions, a portion of code is placed under exception inspection.
- This is done by enclosing that portion of code in a **`try` block**.
    - When an exceptional circumstance arises within that block, an exception is thrown that transfers the control to the exception handler. 
    - If no exception is thrown, the code continues normally and all handlers are ignored.

## Exception Handling Workflow (cont.)

- The exception handler is declared with the **`catch`** keyword immediately after the closing brace of the **`try` block**.
    - The syntax for catch is similar to a regular function with one parameter.
    - The type of this parameter is very important, since the type of the argument passed by the throw expression is checked against it, and only in the case they match, the exception is caught by that handler.    

In [4]:
try
{
    int* myarray= new int[1000000000000];
}
catch (bad_alloc& e)
{
    cout << "This is an exception: " << e.what() << '\n';
}
cout << "Done!";

This is an exception: std::bad_alloc
Done!

## Example: Exception Handling Workflow

- Consider a common arithmetic problem - **division by zero**.
    - Division by zero using integer arithmetic typically causes a program to terminate prematurely.
    - In floating-point arithmetic, many C++ implementations allow division by zero, in which case positive or negative infinity is displayed as **inf** or **-inf**, respectively.
- Typically, a program would simply test for division by zero before attempting the calculation
    - Use exceptions here to present the flow of control when a program executes successfully and when an exception occurs.

## Example: Division By Zero Exception

- Define a function **`quotient`** that receives two integers entered by the user and divides its first `int` parameter by its second `int` parameter.
    - Before performing the division, the function casts the first `int` parameter's value to type `double`.
    - Then, the second `int` parameter's value is promoted to type `double` for the calculation.
    - So function quotient actually performs the division using two `double` values and returns a `double` result.
    
- For the purpose of this example we **treat any attempt to divide by zero as an error**.
    - If the second parameter is zero, the function uses an exception to indicate to the caller that a problem occurred.  

## Example: Division By Zero Exception (cont.)

- For the purpose of this example we **treat any attempt to divide by zero as an error**.
    - Thus, function `quotient` tests its second parameter to ensure that it isn't zero before allowing the division to proceed.
    - If the second parameter is *zero*, the function uses an **exception** to indicate to the caller that a problem occurred.
- The caller can then process the exception and allow the user to type two new values before calling function quotient again.
    - In this way, the program can continue to execute even after an improper value is entered, thus making the program more robust.    

In [5]:
// DivideByZeroException objects should be thrown by functions
// upon detecting division-by-zero exceptions
class DivideByZeroException : public std::runtime_error {
public:
    // constructor specifies default error message
    DivideByZeroException() : runtime_error{"attempted to divide by zero"} {}
};

In [6]:
// perform division and throw DivideByZeroException object if divide-by-zero exception occurs
double quotient(int numerator, int denominator) {
    // throw DivideByZeroException if trying to divide by zero
    if (denominator == 0) {
        throw DivideByZeroException{}; // terminate function
    }
    // return division result
    return static_cast<double>(numerator) / denominator;
}

In [7]:
{
    int number1 = 1, number2 = 0;
    try {
        double result = quotient(number1, number2);
        cout << "The quotient is: " << result << endl;
    }
    catch (const DivideByZeroException& divideByZeroException) {
        cout << "Exception occurred: " << divideByZeroException.what() << endl;
    }
    cout << "Done!";
}

Exception occurred: attempted to divide by zero
Done!

## Exception Class

- A typical exception class that derives from the `runtime_error` class *defines only a constructor* that passes an error-message string to the base-class runtime_error constructor.
- Every exception class that derives directly or indirectly from exception contains the *virtual function* `what`, which returns an exception object's error message.
    - You are not required to derive a custom exception class, such as `DivideByZeroException`, from the standard exception classes provided by C++.
    - Doing so allows you to use the virtual function `what` to obtain an appropriate error message and also allows you to polymorphically process the exceptions by catching a reference to the base-class type.

## `Catch` Handler

- At least one `catch` handler **must immediately follow** each `try` block.
- An exception parameter should always be declared as a **reference** to the type of exception the catch handler can process
    - this prevents copying the exception object when it's caught and allows a catch handler to properly catch derived-class exceptions as well.
- The catch handler that executes is the **first** one whose type **matches the type of the exception** that occurred
- If an exception parameter includes an optional parameter name, the catch handler can use that parameter name to interact with the caught exception in the body of the catch handler

## Termination Model of Exception Handling

1. If an exception occurs as the result of a statement in a `try` block, the `try` block expires (i.e., terminates immediately).
2. Next, the program searches for the first `catch` handler that can process the type of exception that occurred.
3. The program locates the matching catch by comparing the thrown exception's type to each `catch`'s exception-parameter type until the program finds a match.
4. A match occurs if the types are identical or if the thrown exception's type is a derived class of the exception-parameter type.
5. When a match occurs, the code in the matching catch handler executes.

## Termination Model of Exception Handling (cont.)

6. When a `catch` handler finishes processing by reaching its closing right brace (}), the exception is considered handled and the local variables defined within the catch handler (including the catch parameter) go out of scope.
7. Program control does not return to the point at which the exception occurred (known as the throw point), because the `try` block has expired.
8. Rather, control resumes with the **first statement after the last `catch` handler** following the `try` block.

## Termination Model of Exception Handling (cont.)

- This is known as the **termination model of exception handling**.
    - As with any other block of code, when a try block terminates, local variables defined in the block go out of scope.
- If the `try` block completes its execution successfully (i.e., no exceptions occur in the try block), then the program ignores the `catch` handlers and program control continues with the first statement after the last catch following that try block.
- If an *exception that occurs* in a `try` block has no matching `catch` handler, or if an exception occurs in a statement that is not in a try block, the function that contains the statement terminates immediately, and the program attempts to locate an enclosing `try` block in the calling function.

## Rethrowing An Exception

- A function might use a resource - like a file - and might want to release the resource if an exception occurs.
- An exception handler, upon receiving an exception, can release the resource then notify its caller that an exception occurred by rethrowing the exception via the statement
```c++
throw;
```
- Regardless of whether a handler can process an exception, the handler can **rethrow the exception** for further processing outside the handler.
- The next enclosing try block detects the rethrown exception, which a catch handler listed after that enclosing try block attempts to handle.

In [8]:
void throwException() {
    // throw exception and catch it immediately
    try {
        cout << " Function `throwException` throws an exception\n";
        throw exception{}; // generate exception
    }
    catch (const exception&) { // handle exception
        cout << " Exception handled in function `throwException`"
             << "\n Function `throwException` rethrows exception";
        throw; // rethrow exception for further processing
    }
    cout << "This should not print\n";
}

In [9]:
{
    // throw exception
    try {
        cout << "Invoke function throwException\n";
        throwException();
        cout << "This should not print\n";
    }
    catch (const exception&) { // handle exception
        cout << "\n\nException handled in the caller's scope\n";
    }
    cout << "Program control continues after catch\n";
}

Invoke function throwException
 Function `throwException` throws an exception
 Exception handled in function `throwException`
 Function `throwException` rethrows exception

Exception handled in the caller's scope
Program control continues after catch


## Stack Unwinding

- When an exception is thrown but not caught in a particular scope, the function call stack is **"unwound"**, and an attempt is made to catch the exception in the next outer `try...catch` block.
- **Unwinding the function call stack** means that the function in which the exception was not caught terminates, all local variables that have completed intitialization in that function are destroyed and control returns to the statement that originally invoked that function.
    - If a `try` block encloses that statement, an attempt is made to `catch` the exception.
    - If a `try` block **does not enclose** that statement, stack unwinding **occurs again**.
    - If no `catch` handler ever catches this exception, the program terminates.

In [10]:
// function2 invokes throwException
void function2() {
    cout << "`throwException` is called inside `function2`" << endl;
    throwException(); // stack unwinding occurs, return control to function1
    cout << "Done f2";
}

In [11]:
// function1 invokes function2
void function1() {
    cout << "`function2` is called inside `function1`" << endl;
    function2(); // stack unwinding occurs, return control to main
    cout << "Done f1";
}

In [12]:
{
    // invoke function1
    try {
        cout << "`function1` is called" << endl;
        function1(); // call function1 which throws runtime_error
    }
    catch (const exception& error) { // handle runtime error
        cout << "\n\nException occurred: " << error.what() << endl;
        cout << "Exception handled" << endl;
    }
}

`function1` is called
`function2` is called inside `function1`
`throwException` is called inside `function2`
 Function `throwException` throws an exception
 Exception handled in function `throwException`
 Function `throwException` rethrows exception

Exception occurred: std::exception
Exception handled


## Constructors and Exception Handling

- What happens when an error is detected in a **constructor**?
    - Because the constructor **cannot return a value** to indicate an error, we must choose an alternative means of indicating that the object has not been constructed properly.

Error handling:
1. Return the improperly constructed object and hope that anyone using it would make appropriate tests to determine that it's in an inconsistent state.
2. Set some variable outside the constructor.
3. Require the constructor to throw an exception that contains the error information, thus offering an opportunity for the program to handle the failure.


## Destructors and Exception Handling

- If an exception occurs during object construction, destructors may be called:
    - If an exception is thrown before an object is fully constructed, destructors will be called for any member objects that have been constructed so far.
    - If an array of objects has been partially constructed when an exception occurs, only the destructors for the array’s constructed objects will be called.
- Destructors are called for every automatic object constructed in a try block before an exception that occurred in that block is caught.
- Stack unwinding is guaranteed to have been completed at the point that an exception handler begins executing.
- If a destructor invoked as a result of stack unwinding throws an exception, the program terminates.

## Error-Prevention Tip

- Destructors should catch exceptions to prevent program termination.
- Do not throw exceptions from the constructor of a global object or a static local object.Such exceptions cannot be caught, because they're constructed before `main` executes.
- When an exception is thrown from the constructor for an object that's created in a new expression, the dynamically allocated memory for that object is released.
- A constructor should throw an exception if a problem occurs while initializing an object. Before doing so, the constructor should release any memory that it dynamically allocated.

## Standard Library Exception Classes

![Exceptions](../img/Exceptions.jpg)